In [0]:
import tensorflow as tf
import cv2 as cv
import os
import numpy as np
import glob
from random import shuffle

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls '/content/drive/'

'My Drive'


In [0]:

folder="/content/drive/My Drive/A1-Q2_Dataset/HumanActionClassification/"
nls=[]
notp=[]
for i in range(7):
    for filename in glob.glob(folder+str(i+1)+'/*.jpg'):
        img = cv.imread(filename)
        if img is not None:
          img = cv.resize(img, (100, 100))
          nls.append(img)
          notp.append(i)
    

In [0]:
ls=[]
otp=[]
a=[h for h in range(911)]
shuffle(a)
print(a)
for h in a:
  ls.append(nls[h])
  otp.append(notp[h])


In [0]:
ls1=ls[683:]
ls1= np.asarray(ls1,dtype = np.float32)
ls=ls[:683]

In [0]:
ls = np.asarray(ls,dtype = np.float32)
print(ls.shape)

(683, 100, 100, 3)


In [0]:
otp1=otp[683:]
otp1= np.asarray(otp1,dtype = np.int)
otp1=np.eye(7,dtype=int)[otp1]
otp=otp[:683]
otp = np.asarray(otp,dtype =int)


In [0]:
otp=np.eye(7,dtype=int)[otp]
print(otp.shape)

(683, 7)


In [0]:
def conv2d(X,w):
    return tf.nn.conv2d(X,w,strides=[1,1,1,1],padding="SAME")

def maxpool2d(X):
    return tf.nn.max_pool(X,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


In [0]:
def conv_neural_network(X):
  
    weights={'w_conv1':tf.Variable(tf.random_normal([5,5,3,32])),
             'w_conv2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
             'w_fc': tf.Variable(tf.random_normal([25*25*64,128])),
             'out': tf.Variable(tf.random_normal([128,7])),}

    biases={'b_conv1':tf.Variable(tf.random_normal([32])),
             'b_conv2': tf.Variable(tf.random_normal([64])),
             'b_fc': tf.Variable(tf.random_normal([128])),
             'out': tf.Variable(tf.random_normal([7])),}
   
    conv1=conv2d(X,weights['w_conv1'])
    conv1=maxpool2d(conv1)

    conv2 = conv2d(conv1, weights['w_conv2'])
    conv2 = maxpool2d(conv2)
    
    fc1 = tf.reshape(conv2, [-1, weights['w_fc'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['w_fc']), biases['b_fc'])
    fc1 = tf.nn.relu(fc1)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [0]:

def train_neural_network(X):
    n_classes=7
    batch_size=64

    x=tf.placeholder('float',[None,100,100,3])
    y=tf.placeholder('float',[None,7])
    prediction=conv_neural_network(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = prediction,labels=y))
    optimizer=tf.train.AdamOptimizer().minimize(cost)
    
    epochs=100
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for ep in range(epochs):   
            for  j in range(len(ls)//batch_size):
                #b_x=np.zeros((64,100,100,3))
                b_x=ls[j*batch_size:(j*batch_size+batch_size)]
              
                #b_y=np.zeros((64,7))
                b_y=otp[j*batch_size:(j*batch_size+batch_size)]
                correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#                 opt = sess.run(optimizer, feed_dict={x: b_x,y: b_y})
                _,loss, acc = sess.run([optimizer,cost, accuracy], feed_dict={x: b_x,y: b_y})
            print("epoch " + str(ep) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
            print("Optimization Finished!")
            
        
            test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: ls1,y : otp1})
            print("Testing Accuracy:","{:.5f}".format(test_acc))
train_neural_network(ls)

epoch 0, Loss= 34502524.000000, Training Accuracy= 0.17188
Optimization Finished!
Testing Accuracy: 0.14912
epoch 1, Loss= 17613374.000000, Training Accuracy= 0.20312
Optimization Finished!
Testing Accuracy: 0.15351
epoch 2, Loss= 15926286.000000, Training Accuracy= 0.20312
Optimization Finished!
Testing Accuracy: 0.18860
epoch 3, Loss= 12997898.000000, Training Accuracy= 0.28125
Optimization Finished!
Testing Accuracy: 0.17544
epoch 4, Loss= 11187177.000000, Training Accuracy= 0.29688
Optimization Finished!
Testing Accuracy: 0.17982
epoch 5, Loss= 8581988.000000, Training Accuracy= 0.34375
Optimization Finished!
Testing Accuracy: 0.17105
epoch 6, Loss= 7251221.500000, Training Accuracy= 0.34375
Optimization Finished!
Testing Accuracy: 0.18860
epoch 7, Loss= 6597481.000000, Training Accuracy= 0.42188
Optimization Finished!
Testing Accuracy: 0.18860
epoch 8, Loss= 5937952.000000, Training Accuracy= 0.45312
Optimization Finished!
Testing Accuracy: 0.18860
epoch 9, Loss= 5228985.000000, T